# Advanced Examples

In [ ]:
import libfmp.b as lfb
import numpy as np
import cv2

from example_data import *

# This file uses multiple Sessions, but only one will be open at a time, unless MULTI_SESSION is set to True
MULTI_SESSION = False
session_storage = dict()

sr = 48000
audio_a, _ = lfb.read_audio(simple_audio_file, mono=True, Fs=sr)

def load_av_example_data():
    WIDTH = 48 * 10
    HEIGHT = 36 * 10

    def frame2array(f):
        img = cv2.cvtColor(f, cv2.COLOR_BGR2GRAY)
        img = cv2.resize(img, (WIDTH, HEIGHT))
        return np.array(img, dtype=np.uint8)

    video_capture = cv2.VideoCapture(video_file)
    arrays = list()
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        array = frame2array(frame)
        arrays.append(array)

    array = np.stack(arrays, axis=0)

    audio, _ = lfb.read_audio(audio_to_video_file, Fs=sr, mono=True)

    return audio, array


audio_b, video = load_av_example_data()

In [ ]:
# advanced 1:
# this example shows navigation between multiple audio files in one Session
# it will loop and alternate between files
# the time encodes the specific version
# all versions are mapped linearly (in a real application these mappings would be more complicated)

from makeplotplayable import Session

# combine audio clips into one to allow Session to play it
combined_audio = np.concatenate((audio_a, audio_b))

session = Session(combined_audio, sr, looping=True)
session.start()


# noinspection PyShadowingNames
@session
def plot(x, sr, durations, index, name):
    import numpy as np
    import libfmp.b as lfb

    start_times = np.zeros_like(durations)
    for i in range(durations.shape[0] - 1):
        start_times[i + 1] = start_times[i] + durations[i]

    fig, ax, line = lfb.plot_signal(x, sr)

    # map the time of all clips to a position
    def time_to_pos(time):
        j = np.searchsorted(start_times, time) - 1
        time -= start_times[j]
        progress = time / durations[j]
        pos = progress * durations[index]
        return pos

    return fig, ax, {"title": name,
                     "custom_time_to_pos_function": time_to_pos,
                     "custom_pos_to_time_function": lambda pos: pos + start_times[
                         index]}  # map to time depending on start time


durations = np.array([audio_a.shape[0] / sr, audio_b.shape[0] / sr])

# it is possible to use different plotting functions
plot(audio_a, sr, durations, 0, 'A')
plot(audio_b, sr, durations, 1, 'B')

if MULTI_SESSION:
    session_storage["advanced 1"] = session

In [ ]:
# advanced 2:
# this example shows that animating a plot even allows for video playback

from makeplotplayable import Session

session = Session(audio_b, sr, looping=True)
session.start()


# noinspection PyShadowingNames
@session
def plot_audio(x, sr):
    import libfmp.b as lfb

    fig, ax, line = lfb.plot_signal(x, sr)

    #return the figure and the axis for the cursor as a tuple
    return fig, ax, {"title": "Audio for Video"}


plot_audio(audio_b, sr)


# noinspection PyShadowingNames
@session
def plot_video(video):
    import matplotlib.pyplot as plt
    import numpy as np

    fig = plt.figure()
    ax = plt.subplot(1, 1, 1)

    mat_plot = ax.imshow(np.zeros_like(video[0]), cmap='gray', vmin=0, vmax=255, interpolation='antialiased')

    last_index = 0

    # draw the correct frame (pos is frame index)
    def draw_function(time, pos, paused):
        nonlocal last_index
        # don't redraw if the frame has not changed
        if last_index == pos:
            return False

        mat_plot.set(data=video[pos])
        last_index = pos
        return True

    # implement play/pause functionality
    pp_pressed = False

    def on_key(event):
        nonlocal pp_pressed
        if event.key in [" ", "enter"]:
            pp_pressed = True

    fig.canvas.mpl_connect('key_press_event', on_key)

    def update_func(time, pos, paused):
        nonlocal pp_pressed
        new_paused = paused ^ pp_pressed
        pp_pressed = False
        return time, new_paused

    return fig, ax, {"title": "Video",
                     "draw_function": draw_function,
                     "artists": [mat_plot],
                     "override_update_function": update_func,  # implement play/pause functionality + disable cursor
                     "custom_time_to_pos_function": lambda time: int(
                         min(video.shape[0] - 1, time * 30))}  # calculate the frame index


plot_video(video)

if MULTI_SESSION:
    session_storage["advanced 2"] = session

In [ ]:
# advanced 3:
# Example to test streaming

from makeplotplayable import Session

session = Session.from_file(long_audio_file)
session.start()


# noinspection PyShadowingNames
@session
def plot(duration):
    import matplotlib.pyplot as plt
    import numpy as np

    fig, ax = plt.subplots()

    ax.plot(np.array([0, duration]))

    return fig, ax, {"title": "Streaming Example"}


plot(session.duration)

if MULTI_SESSION:
    session_storage["advanced 3"] = session

In [ ]:
import time
# advanced 4:
# Stereo Test

from makeplotplayable import Session

combined_audio = np.stack((audio_a[:sr * 10], audio_b[:sr * 10]), axis=0)

session = Session(combined_audio, sr)
session.start()


# noinspection PyShadowingNames
@session
def plot(audio, sr, title):
    import libfmp.b as lfb

    fig, ax, line = lfb.plot_signal(audio, sr)

    return fig, ax, {"title": title}


plot(combined_audio[0], sr, "Left")
plot(combined_audio[1], sr, "Right")

time.sleep(10)

if MULTI_SESSION:
    session_storage["advanced 4"] = session